Escolher uma API da lista disponível; 
Fazer a extração de pelo menos 3 tabelas;
Criar um alerta de erro usando a função alerta() do desafio da aula Fundamentos de programação III.

In [126]:
import pandas as pd
import requests 
import os
import time 

In [127]:
url1 = "https://brasilapi.com.br/api/banks/v1"
url2 = "https://brasilapi.com.br/api/cvm/corretoras/v1"
url3 = "https://brasilapi.com.br/api/pix/v1/participants"

In [128]:
# Criando uma função para não ficar repetindo a bloco de notificação
def notificacao(url, title):
    try:
        response = requests.get(url)
        
        if response.status_code == 200:
            mensagem = f"Conexão bem-sucedida - Status {response.status_code}"
            data_json = response.json()
        else:
            mensagem = f"Falha na conexão - Erro {response.status_code}"
            data_json = None #significa vazio...

    except :
        None
    
    os.system(f'''osascript -e 'display notification "{mensagem}" with title "{title}"' ''')
    return data_json

data_json1 = notificacao(url1, "Resultado API - 1")
time.sleep(3)
data_json2 = notificacao(url2, "Resultado API - 2")
time.sleep(3)
data_json3 = notificacao(url3, "Resultado API - 3")


In [129]:
df_banks = pd.DataFrame(data_json1)
df_banks

,ispb,name,code,fullName
0,00000000,BCO DO BRASIL S.A.,1.0,Banco do Brasil S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70.0,BRB - BANCO DE BRASILIA S.A.
2,00038121,Selic,NaN,Banco Central do Brasil - Selic
3,00038166,Bacen,NaN,Banco Central do Brasil
4,00122327,SANTINVEST S.A. - CFI,539.0,"SANTINVEST S.A. - CREDITO, FINANCIAMENTO E INV..."
...,...,...,...,...
354,92875780,WARREN CVMC LTDA,371.0,WARREN CORRETORA DE VALORES MOBILIÁRIOS E CÂMB...
355,92894922,BANCO ORIGINAL,212.0,Banco Original S.A.
356,94968518,EFX CC LTDA.,289.0,EFX CORRETORA DE CÂMBIO LTDA.
357,,NaN,NaN,NaN


In [130]:
df_banks.duplicated().value_counts() #contando quantas linhas tem duplicadas (se tiver)

False    358
True       1
Name: count, dtype: int64

In [131]:
df_banks[df_banks.duplicated(keep=False)] 

,ispb,name,code,fullName
357,,NaN,NaN,NaN
358,,NaN,NaN,NaN


In [132]:
# Remove as linhas onde tem as 3 colunas nulas
df_banks = df_banks[~(df_banks["code"].isnull() & df_banks["name"].isnull() & df_banks["fullName"].isnull())]

In [133]:
df_banks[df_banks.duplicated(keep=False)] 

,ispb,name,code,fullName


In [134]:
# Como apresentamos alguns codigos ('code') nulos nessa coluna com formato object, estou padronizando como 0.0
df_banks.loc[:, "code"] = df_banks["code"].fillna(0.0)

In [135]:
# Da mesma forma para para a coluna ('ispb'), padronizando como com 8 digitos, pois se trata de um identificador de pagamentos.
# Verificando se existe esse código
print(df_banks[df_banks["ispb"] == '10101010'])

Empty DataFrame
Columns: [ispb, name, code, fullName]
Index: []


In [136]:
# Padronizando com 10101010
df_banks.loc[:, "ispb"] = df_banks["ispb"].fillna(10101010)

# Deixando as strings maiúsculas
df_banks.loc[:, "name"] = df_banks["name"].str.upper()
df_banks.loc[:, "fullName"] = df_banks["fullName"].str.upper()

# Ordenando o DataFrame em ordem crescente dos códigos
df_banks = df_banks.sort_values("code", ascending=True)


In [137]:
# print(df_banks[df_banks["name"].isnull()]) # teste para ver se tem nulo nos nomes
# print(df_banks[df_banks["fullName"].isin([''])]) # teste para ver se tem um 'nome' nulo
df_banks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 357 entries, 184 to 33
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ispb      357 non-null    object 
 1   name      357 non-null    object 
 2   code      357 non-null    float64
 3   fullName  357 non-null    object 
dtypes: float64(1), object(3)
memory usage: 13.9+ KB


In [138]:
df_corretoras = pd.DataFrame(data_json2)
df_corretoras.head(4)

,cnpj,type,nome_social,nome_comercial,status,email,telefone,cep,pais,uf,municipio,bairro,complemento,logradouro,data_patrimonio_liquido,valor_patrimonio_liquido,codigo_cvm,data_inicio_situacao,data_registro
0,76621457000185,CORRETORAS,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,controle@4um.com.br,33519966,80420210,BRASIL,PR,CURITIBA,CENTRO,4º ANDAR,R. VISCONDE DO RIO BRANCO 1488,2005-12-31,4228660.18,2275,2006-10-05,1968-01-15
1,33817677000176,CORRETORAS,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL DTVM,CANCELADA,complianceregulatorio@abcbrasil.com.br,31702172,1453000,,SP,SÃO PAULO,ITAIM BIBI,2º ANDAR,"AV. CIDADE JARDIM, 803",2002-12-31,0.00,3514,2002-10-14,2002-10-14
2,10664027000132,CORRETORAS,ABERTURA CCVM LTDA,ABERTURA CCVM LTDA,CANCELADA,,,50010240,BRASIL,PE,RECIFE,,,R DO IMP.D.PEDRO II 239/CJ.102,1989-12-31,5995252.29,329,1990-06-12,1986-07-08
3,44527444000155,CORRETORAS,ABN AMRO CLEARING CORRETORA DE T-TULOS E VALOR...,,EM FUNCIONAMENTO NORMAL,canal.comunicacao@br.abnamro.com,30737417,4551010,,SP,SÃO PAULO,VILA OLIMPIA,"11¦ ANDAR, CJ. 111","RUA FIDÛNCIO RAMOS, N¦ 302, TORRE B",2023-12-31,11122779.75,3794,2022-06-20,2022-06-20


In [139]:
df_corretoras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   cnpj                      374 non-null    object
 1   type                      374 non-null    object
 2   nome_social               374 non-null    object
 3   nome_comercial            374 non-null    object
 4   status                    374 non-null    object
 5   email                     374 non-null    object
 6   telefone                  374 non-null    object
 7   cep                       374 non-null    object
 8   pais                      374 non-null    object
 9   uf                        374 non-null    object
 10  municipio                 374 non-null    object
 11  bairro                    374 non-null    object
 12  complemento               374 non-null    object
 13  logradouro                374 non-null    object
 14  data_patrimonio_liquido   

In [140]:
distinct_count = df_corretoras["type"].nunique()
distinct_count

1

In [141]:
# Como temos uma coluna com apenas uma informação com nomenclatura 'CORRETORAS', não faz sentido, pois se trata de uma base disso apenas.
df_corretoras.drop(columns=["type"], inplace=True) # o inplace está atribuindo o drop no proprio datrafme


In [142]:
def conversor_datetime(df, colunas):
    for coluna in colunas:
        df[coluna] = pd.to_datetime(df[coluna], errors='coerce').dt.date #errors='coerce' faz com parametros que não são alterados fiquem nulos.
    return df

colunas_conversor_datetime = ['data_patrimonio_liquido', 'data_inicio_situacao', 'data_registro']
df_corretoras = conversor_datetime(df_corretoras, colunas_conversor_datetime)

In [143]:
def conversor_int64(df, colunas):
    for coluna in colunas:
        df[coluna] = pd.to_numeric(df[coluna], errors='coerce').astype('Int64')  
    return df

colunas_conversor_int64 = ['cnpj','telefone','cep','codigo_cvm']
df_corretoras = conversor_int64(df_corretoras,colunas_conversor_int64)

In [144]:
print(df_corretoras.dtypes)

cnpj                         Int64
nome_social                 object
nome_comercial              object
status                      object
email                       object
telefone                     Int64
cep                          Int64
pais                        object
uf                          object
municipio                   object
bairro                      object
complemento                 object
logradouro                  object
data_patrimonio_liquido     object
valor_patrimonio_liquido    object
codigo_cvm                   Int64
data_inicio_situacao        object
data_registro               object
dtype: object


In [145]:
# Validando os Status da base
print(df_corretoras["status"].unique())

['CANCELADA' 'EM FUNCIONAMENTO NORMAL' 'LIQUIDAÇÃO EXTRAJUDICIAL'
 'INCORPORAÇÃO']


In [146]:
# Criando um dataframe apenas de canceladas para alguma analise posterior
df_corretoras_canceladas = df_corretoras[df_corretoras['status'] == 'CANCELADA']

# Removendo os cancelados do dataframe anterior
df_corretoras = df_corretoras[df_corretoras['status'] != 'CANCELADA']


In [147]:
df_pix = pd.DataFrame(data_json3)
df_pix.head(10)

,ispb,nome,nome_reduzido,modalidade_participacao,tipo_participacao,inicio_operacao
0,00000000,BANCO DO BRASIL S.A.,BCO DO BRASIL S.A.,PDCT,DRCT,2020-11-03T09:30:00.000Z
1,00000208,BRB - BANCO DE BRASILIA S.A.,BRB - BCO DE BRASILIA S.A.,PDCT,DRCT,2020-11-03T09:30:00.000Z
2,00068987,COOPERATIVA DE CRÉDITO ARACREDI LTDA. - SICOOB...,CC ARACREDI LTDA.,PDCT,IDRT,2020-11-03T09:30:00.000Z
3,00075847,COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...,CC UNICRED CENTRO-SUL LTDA - UNICRED CENTRO-SUL,PDCT,IDRT,2020-11-03T09:30:00.000Z
4,00106180,CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...,"CCC DOS EST DE MT, MS E MUN DE CACOAL/RO",PDCT,IDRT,2020-11-03T09:30:00.000Z
5,00204963,COOPERATIVA DE CREDITO RURAL SEARA - CREDISEARA,CCR SEARA,PDCT,DRCT,2022-02-11T13:00:00.000Z
6,00259231,SICOOB UNIMAIS METROPOLITANA - COOPERATIVA DE ...,CCLA SICOOB UNIMAIS METROPOLITANA,PDCT,IDRT,2020-11-03T09:30:00.000Z
7,00309024,CENTRAL DAS COOPERATIVAS DE ECONOMIA E CREDITO...,CCC ESTADO MG - CECREMGE,PDCT,IDRT,2020-11-03T09:30:00.000Z
8,00315557,COOPERATIVA CENTRAL DE CRÉDITO UNICRED DO BRAS...,UNICRED DO BRASIL,PDCT,DRCT,2020-11-03T09:30:00.000Z
9,00329598,SEFER INVESTIMENTOS DISTRIBUIDORA DE TÍTULOS E...,SEFER INVESTIMENTOS DTVM LTDA,PDCT,DRCT,2022-11-08T13:00:00.000Z


In [148]:
df_pix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 862 entries, 0 to 861
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ispb                     862 non-null    object
 1   nome                     862 non-null    object
 2   nome_reduzido            862 non-null    object
 3   modalidade_participacao  862 non-null    object
 4   tipo_participacao        862 non-null    object
 5   inicio_operacao          862 non-null    object
dtypes: object(6)
memory usage: 40.5+ KB


In [149]:
#Truncando os 10 primeiros caracteres da coluna inicio_operacao
df_pix['inicio_operacao'] = df_pix['inicio_operacao'].astype(str).str[:10]

In [150]:
df_pix = conversor_datetime(df_pix, ['inicio_operacao'])
df_pix = conversor_int64(df_pix, ['ispb'])


In [151]:
df_pix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 862 entries, 0 to 861
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ispb                     862 non-null    Int64 
 1   nome                     862 non-null    object
 2   nome_reduzido            862 non-null    object
 3   modalidade_participacao  862 non-null    object
 4   tipo_participacao        862 non-null    object
 5   inicio_operacao          862 non-null    object
dtypes: Int64(1), object(5)
memory usage: 41.4+ KB


In [152]:
df_pix.head()

,ispb,nome,nome_reduzido,modalidade_participacao,tipo_participacao,inicio_operacao
0,0,BANCO DO BRASIL S.A.,BCO DO BRASIL S.A.,PDCT,DRCT,2020-11-03
1,208,BRB - BANCO DE BRASILIA S.A.,BRB - BCO DE BRASILIA S.A.,PDCT,DRCT,2020-11-03
2,68987,COOPERATIVA DE CRÉDITO ARACREDI LTDA. - SICOOB...,CC ARACREDI LTDA.,PDCT,IDRT,2020-11-03
3,75847,COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...,CC UNICRED CENTRO-SUL LTDA - UNICRED CENTRO-SUL,PDCT,IDRT,2020-11-03
4,106180,CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...,"CCC DOS EST DE MT, MS E MUN DE CACOAL/RO",PDCT,IDRT,2020-11-03


In [153]:
import sqlite3

In [154]:
# Conexão ao banco de dados SQLite
conn = sqlite3.connect('tabela_projeto_final_1.db')

# Inserir os DataFrames
df_pix.to_sql('pix', conn, if_exists='replace', index=False)
df_banks.to_sql('banks', conn, if_exists='replace', index=False)
df_corretoras.to_sql('corretoras', conn, if_exists='replace', index=False)
df_corretoras_canceladas.to_sql('corretoras_canceladas', conn, if_exists='replace', index=False)

# Fechar a conexão
conn.close()

In [155]:
# testando conexão
conn = sqlite3.connect('tabela_projeto_final_1.db')
query = ''' SELECT * FROM pix'''

df_teste = pd.read_sql(query,conn)
conn.close()
df_teste.head(3)

,ispb,nome,nome_reduzido,modalidade_participacao,tipo_participacao,inicio_operacao
0,0,BANCO DO BRASIL S.A.,BCO DO BRASIL S.A.,PDCT,DRCT,2020-11-03
1,208,BRB - BANCO DE BRASILIA S.A.,BRB - BCO DE BRASILIA S.A.,PDCT,DRCT,2020-11-03
2,68987,COOPERATIVA DE CRÉDITO ARACREDI LTDA. - SICOOB...,CC ARACREDI LTDA.,PDCT,IDRT,2020-11-03
